# TechBytes: Using Python with Teradata Vantage
## Part 4: In-Database scripting with the SCRIPT Table Operator - Map functions

The contents of this file are Teradata Public Content and have been released to the Public Domain.
Please see _license.txt_ file in the package for more information.

Alexander Kolovos and Tim Miller - May 2021 - v.2.0 \
Copyright (c) 2021 by Teradata \
Licensed under BSD

This TechByte demonstrates a few different ways in which teradataml helps you use the SCRIPT Table Operator (STO) Database object for native execution of Python scripts inside the Database. Specifically, this TechByte shows how to
* test a Python script on the client for correct in-Database execution with the teradataml STO Sandbox feature.
* bring into the Database a Python model you have previously trained, and use it with a scoring script for in-Database scoring.
* scale in-Database tasks of training and scoring with multiple models when using partitioned data.
* use the teradataml DataFrame Map functions; namely, use the map_row() method for row-based operations and the map_partition() method for partition-based operations.

_Note_: To perform in-nodes script execution, you need to coordinate with your Database Administrator (DBA), and ensure that (a) the STO Database object is activated in the Advanced SQL Engine, (b) the Teradata In-Nodes Python Interpreter and Add-ons packages are installed in the target server, and (c) your Database user account has the necessary STO permissions enabled by the DBA.

Contributions by:
- Alexander Kolovos, Sr Staff Software Architect, Teradata Product Engineering / Vantage Cloud and Applications.
- Tim Miller, Principal Software Architect, Teradata Product Management / Advanced Analytics.

### Initial Steps: Load libraries and create a Vantage connection

In [2]:
# Load teradataml and dependency packages.
#
import os
import getpass as gp

from teradataml import create_context, remove_context, get_context
from teradataml import DataFrame, copy_to_sql, in_schema
from teradataml.options.display import display

from teradataml.table_operators.Script import Script
from teradataml.table_operators.sandbox_container_util import *
from teradatasqlalchemy import (VARCHAR, INTEGER, FLOAT, CLOB)

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pickle
import base64

In [3]:
# Specify a Teradata Vantage server to connect to. In the following statement, 
# replace the following argument values with strings as follows:
# <HOST>   : Specify your target Vantage system hostname (or IP address).
# <UID>    : Specify your Database username.
# <PWD>    : Specify your password. You can also use encrypted passwords via
#            the Stored Password Protection feature.
#con = create_context(host = <HOST>, username = <UID>, password = <PWD>, 
#                     database = <DB_Name>, "temp_database_name" = <Temp_DB_Name>)
#
con = create_context(host = "tdprd.td.teradata.com", username = "ak186064",
                            password = gp.getpass(prompt='Password:'), 
                            logmech = "LDAP", database = "TRNG_TECHBYTES",
                            temp_database_name = "ak186064")

Password: ··············


In [4]:
# Create a teradataml DataFrame from the ADS we need, and take a glimpse at it.
#
td_ADS_Py = DataFrame("ak_TBv2_ADS_Py")
td_ADS_Py.to_pandas().head(5)

,cust_id,income,age,tot_cust_years,tot_children,female_ind,single_ind,married_ind,separated_ind,state_code,...,ck_avg_bal,sv_avg_bal,cc_avg_bal,ck_avg_tran_amt,sv_avg_tran_amt,cc_avg_tran_amt,q1_trans_cnt,q2_trans_cnt,q3_trans_cnt,q4_trans_cnt
0,28617939,5724.2,70,10,1,1,0,1,0,CA,...,291.728969,291.728969,291.728969,2.879832,2.879832,2.879832,384,57,69,72
1,27252140,48900.0,44,6,1,0,0,1,0,OTHER,...,4875.089945,0.000000,4875.089945,14.125061,14.125061,14.125061,220,28,30,48
2,23166920,31020.6,73,9,0,0,0,1,0,CA,...,6059.145000,0.000000,0.000000,-28.884351,-28.884351,0.000000,15,14,17,11
3,19077268,6445.6,66,6,1,1,0,1,0,NY,...,4281.467000,0.000000,0.000000,-5.011717,-5.011717,0.000000,54,45,0,0
4,23168535,1625.2,39,5,0,1,0,1,0,TX,...,1430.087142,1430.087142,0.000000,41.337208,41.337208,0.000000,80,100,58,22


In [5]:
# Split the ADS into 2 samples, each with 60% and 40% of total rows.
# Use the 60% sample to train, and the 40% sample to test/score.
# Persist the samples as tables in the Database.
#
td_Train_Test_ADS = td_ADS_Py.sample(frac = [0.6, 0.4])

Train_ADS = td_Train_Test_ADS[td_Train_Test_ADS.sampleid == "1"]
copy_to_sql(Train_ADS, table_name="ak_TBv2_Train_ADS_Py", if_exists="replace")

Test_ADS = td_Train_Test_ADS[td_Train_Test_ADS.sampleid == "2"]
copy_to_sql(Test_ADS, table_name="ak_TBv2_Test_ADS_Py", if_exists="replace")

### 1. BYOM: In-Database scoring with external Python model
In this segment, we illustrate training a Random Forests classification model on the client, and saving it as a binary file. Eventually, we send the model together with a scoring script to the Database, and use them in the Database environment to scale a scoring task of a test dataset. In between, we demonstrate testing of the scoring script outside the Database in the teradataml STO Sandbox.

#### 1.1. Random Forests classification model training on the client

In [6]:
# Start with the training subset of the ADS. First, read it from the table.
# Then, convert to pandas DataFrame to enable subsequent modeling operations.
#
td_Train_ADS = DataFrame("ak_TBv2_Train_ADS_Py")

df_Train_ADS = td_Train_ADS.to_pandas()
df_Train_ADS.head()

,cust_id,income,age,tot_cust_years,tot_children,female_ind,single_ind,married_ind,separated_ind,state_code,...,sv_avg_bal,cc_avg_bal,ck_avg_tran_amt,sv_avg_tran_amt,cc_avg_tran_amt,q1_trans_cnt,q2_trans_cnt,q3_trans_cnt,q4_trans_cnt,sampleid
0,28617939,5724.2,70,10,1,1,0,1,0,CA,...,291.728969,291.728969,2.879832,2.879832,2.879832,384,57,69,72,1
1,27252140,48900.0,44,6,1,0,0,1,0,OTHER,...,0.000000,4875.089945,14.125061,14.125061,14.125061,220,28,30,48,1
2,25889191,5828.1,69,6,0,1,0,1,0,OTHER,...,3568.918000,0.000000,0.000000,0.000000,0.000000,7,9,7,9,1
3,21806880,0.0,17,5,1,0,1,0,0,TX,...,829.489960,829.489960,0.000000,0.000000,37.680260,24,10,6,10,1
4,23168535,1625.2,39,5,0,1,0,1,0,TX,...,1430.087142,0.000000,41.337208,41.337208,0.000000,80,100,58,22,1


In [7]:
# Determine the columns that the predictor accounts for.
#
predictor_columns = ["income", "age", "tot_cust_years", "tot_children",
                     "female_ind", "single_ind", "married_ind", "separated_ind",
                     "ck_acct_ind", "sv_acct_ind", "ck_avg_bal", "sv_avg_bal",
                     "ck_avg_tran_amt", "sv_avg_tran_amt", "q1_trans_cnt",
                     "q2_trans_cnt", "q3_trans_cnt", "q4_trans_cnt"]

# Note: At time of creation of this TechByte, in-nodes Python has RF classifier
#       from the scikit-learn add-on library v.0.22.2.post1. Keep an eye for  
#       potential incompatibilites, if a package version on your client should
#       differ from the in-nodes add-on version. 
#       In the present TechByte, the client carries scikit-learn add-on library
#       v.0.23.2. No issues were observed when using a model built with this 
#       later version for in-nodes scoring.
#       In case errors may be produced due to different scikit-learn versions 
#       on the client and in-nodes, you can try switching your client's add-on
#       scikit-learn version to match in-nodes by using the explicit command:
#       "pip install scikit-learn==<version>"

# For the classifier, specify the following parameters:
# ntree: n_estimators=500, mtry: max_features=5, nodesize: min_samples_leaf=1 (default; skipped)
#
classifier = RandomForestClassifier(n_estimators=500, max_features=5, random_state=0)
X = df_Train_ADS[predictor_columns]
y = df_Train_ADS["cc_acct_ind"]

# Train the Random Forest model to predict Credit Card account ownership based upon specified independent variables.
#
classifier = classifier.fit(X, y)

print("Model training complete.")

Model training complete.


In [8]:
# Save model into file in present client folder where the notebook executes.
# Note: In the following, we are using both pickle (to serialize) and base64
#       (to encode) the model prior to saving it into a file. If model is only
#       pickled, then unplickling in the Database might produce a pickle
#       AttributeError that claims an "X object has no attribute Y". This is 
#       related to namespaces in client and target systems. See more info at: 
#       https://docs.python.org/3/library/pickle.html#pickling-class-instances
#
filePath = "./Inputs/"
modelFileName = "RFmodel_py.out"
classifierPkl = pickle.dumps(classifier)
classifierPklB64 = base64.b64encode(classifierPkl)
with open(filePath + modelFileName, 'wb') as fOut:            # Write in binary format
    fOut.write(classifierPklB64)

print("Model saved in file '" + filePath + modelFileName + "'.")

Model saved in file './Inputs/RFmodel_py.out'.


#### 1.2. Setting up the teradataml Script object and STO Sandbox

In [9]:
# Here is the path where we keep necessary files for this demo.
#
path_to_files = "./Inputs/"
#
# Request to print the SQL submitted to the Advanced SQL Engine.
#
display.print_sqlmr_query = True

# Set SQL SEARCHUIFDBPATH to database where script-related files are installed.
#
con.execute("SET SESSION SEARCHUIFDBPATH = TRNG_TECHBYTES;") 

In [10]:
# Specify the teradataml DataFrame to use with the teradataml Script object.
#
td_Test_ADS = DataFrame("ak_TBv2_Test_ADS_Py")

# Recall that the testing subset table has an additional column with the sample
# ID. We drop it because our script does not account for the extra column.
td_Test_ADS = td_Test_ADS.drop(columns = "sampleid")

# To test the script in the STO Sandbox, we would like to have a sample of
# the test dataset handy for use in the STO Sandbox. To this end, we bring 500
# rows of the test dataset from the Database, and save them into a csv file.
# Note: Exclude the DataFrame index from the csv file; this prevents the index
#       from being assumed to be an additional data column in the csv file. 
#
df_Test_ADS_Sample = td_Test_ADS.to_pandas(num_rows = 500)
df_Test_ADS_Sample.to_csv(path_to_files + "stoSandboxTestData.csv", index = False)
df_Test_ADS_Sample.head(5)

,cust_id,income,age,tot_cust_years,tot_children,female_ind,single_ind,married_ind,separated_ind,state_code,...,ck_avg_bal,sv_avg_bal,cc_avg_bal,ck_avg_tran_amt,sv_avg_tran_amt,cc_avg_tran_amt,q1_trans_cnt,q2_trans_cnt,q3_trans_cnt,q4_trans_cnt
0,14995552,10575.4,67,7,2,1,0,1,0,NY,...,0.000000,0.000000,4900.000000,0.000000,0.000000,-427.963607,0,0,10,18
1,21801248,0.0,17,5,1,1,1,0,0,OTHER,...,1769.340739,1769.340739,1769.340739,-1.150956,-1.150956,-1.150956,183,153,120,69
2,25889191,5828.1,69,6,0,1,0,1,0,OTHER,...,0.000000,3568.918000,0.000000,0.000000,0.000000,0.000000,7,9,7,9
3,19084842,115656.8,57,7,2,0,0,1,0,IL,...,4414.437899,4414.437899,0.000000,0.979712,0.979712,0.000000,180,32,18,48
4,16353660,971.5,81,11,3,1,0,0,1,NY,...,192.862197,192.862197,192.862197,-0.273506,-0.273506,-0.273506,363,69,60,87


In [11]:
# Define a teradataml Script object. We will use this object with calls
# to the STO Sandbox for testing and validation of the Python scoring script
# before the script is sent to the Database.
# Note: In present use case, the Python script will be importing a model file.
#       Remember to adjust the relative model file location in the Python code.
#       In the STO Sandbox, the model is expected to be found in the same,
#       current Sandbox directory where the script is placed, too.
#
stoSB = Script(data = td_Test_ADS,
               script_name = "stoRFScoreSB.py",
               files_local_path = path_to_files, 
               script_command = "python3 ./TRNG_TECHBYTES/stoRFScoreSB.py",
               delimiter = ',',
               returns = { "ID": INTEGER(), "Prob_0": FLOAT(), "Prob_1": FLOAT(), "Actual": INTEGER() }
              )

In [12]:
# Set up the STO Sandbox with the Python STO Sandbox Docker image available
# at downloads.teradata.com. Currently, when specifying the sandbox by image
# location, then the "sandbox_image_name" must be specified, too, and must be
# "stosandbox:1.0".
#
sb_path = "/Users/ak186064/Documents/Projects/TD-Team/Proj_STO_Sandbox/"
setup_sandbox_env(sandbox_image_location = sb_path + "sto_sandbox_Python3.7.7_sles12sp3.0.5.4_docker_image.1.0.0.tar.gz",
                  sandbox_image_name = "stosandbox:1.0")

configure.sandbox_container_id

Loading image from /Users/ak186064/Documents/Projects/TD-Team/Proj_STO_Sandbox/sto_sandbox_Python3.7.7_sles12sp3.0.5.4_docker_image.1.0.0.tar.gz. It may take few minutes.
Image loaded successfully.
Starting a container for stosandbox:1.0 image.
Container a6b527adbe596e93d1d9f98033a515cf14a9d09e11ab666138926d4d0bc6c3e7 started successfully.


'a6b527adbe596e93d1d9f98033a515cf14a9d09e11ab666138926d4d0bc6c3e7'

#### 1.3. Script code testing in the STO Sandbox

In [13]:
testOut = stoSB.test_script(input_data_file = "stoSandboxTestData.csv",
                            supporting_files = "RFmodel_py.out"
                           )
testOut.head(n = 5)

,ID,Prob_0,Prob_1,Actual
0,14995552,0.000,1.000,1
1,21801248,0.000,1.000,1
2,25889191,0.996,0.004,0
3,19084842,0.796,0.204,0
4,16353660,0.000,1.000,1


In [14]:
# Clean up the STO sandbox
#
cleanup_sandbox_env()

Removed container: a6b527adbe596e93d1d9f98033a515cf14a9d09e11ab666138926d4d0bc6c3e7

Removed image: stosandbox:1.0


#### 1.4. Script code execution in the Database with the STO

In [15]:
# Define a different teradataml Script object. We will use this object with
# calls to the STO for execution of the Python scoring script in-Database.
# Note: In present use case, the Python script will be importing a model file.
#       Remember to adjust the relative model file location in the Python code.
#       In the Database, the model file is expected to be found inside the
#       node directory that is named after the SEARCHUIFDBPATH.
# In general, there is no need to define different teradataml Script objects
# when the script code is identical for Sandbox and in-Database use.
#
sto = Script(data = td_Test_ADS,
             script_name = "stoRFScore.py",
             files_local_path = path_to_files, 
             script_command = "python3 ./TRNG_TECHBYTES/stoRFScore.py",
             delimiter = ',',
             returns = { "ID": INTEGER(), "Prob_0": FLOAT(), "Prob_1": FLOAT(), "Actual": INTEGER() }
            )

In [16]:
# If previous versions of files exist in the Database, remove them prior to
# installing the current versions you wish to use.
#
sto.remove_file(file_identifier='RFmodel_py', force_remove=True)
sto.remove_file(file_identifier='stoRFScore', force_remove=True)

# Install the script and the accompanying model file in the target Advanced SQL
# Engine Database. Remember to specify in your script code the correct path to
# the model file in the Database: Your script will be looking for the model
# file in a node directory named after the SEARCHUIFDBPATH.
#
sto.install_file(file_identifier='RFmodel_py', file_name='RFmodel_py.out', is_binary=True)
sto.install_file(file_identifier='stoRFScore', file_name='stoRFScore.py', is_binary=False)

File RFmodel_py removed from Vantage
File stoRFScore removed from Vantage
File RFmodel_py.out installed in Vantage
File stoRFScore.py installed in Vantage


In [17]:
# Execute the script in-Database with the SCRIPT Table Operator.
#
sto.execute_script()

SELECT * FROM Script(
	ON "ak186064"."ml__select__1620181958646633" AS "input"
	PARTITION BY ANY
	
	SCRIPT_COMMAND('python3 ./TRNG_TECHBYTES/stoRFScore.py')
	delimiter(',')
	returns('ID INTEGER, Prob_0 FLOAT, Prob_1 FLOAT, Actual INTEGER')
) as sqlmr


         ID  Prob_0  Prob_1  Actual
0  21814288   0.002   0.998       1
1  28627788   0.938   0.062       0
2  21803888   0.016   0.984       1
3  23172088   1.000   0.000       0
4  29981226   0.990   0.010       0
5  23178905   0.450   0.550       0
6  29991940   0.864   0.136       0
7  14998170   0.062   0.938       1
8  27251820   0.008   0.992       1
9  19075924   0.006   0.994       1

### 2. Micromodeling: Scaled, In-Database training and scoring of multiple models
When you need to train a different model for each value of a feature and then score corresponding data, Vantage and teradataml can help you scale the entire operation by training and scoring multiple models in parallel in the Advanced SQL Engine.

In [18]:
# Here is the path where we keep necessary files for this demo.
#
path_to_files = "./Inputs/"
#
# Request to print the SQL submitted to the Advanced SQL Engine.
#
display.print_sqlmr_query = True

# Set SQL SEARCHUIFDBPATH to database where script-related files are installed.
#
con.execute("SET SESSION SEARCHUIFDBPATH = TRNG_TECHBYTES;") 

#### 2.1. Models training with the STO

In [19]:
# Step 1: Model training
# 
# Start with the training subset of the ADS. First, read it from the table.
# Then, convert to pandas DataFrame to enable subsequent modeling operations.
#
td_Train_ADS = DataFrame("ak_TBv2_Train_ADS_Py")

# Recall that the training subset table has an additional column of the sample
# ID. We drop it because our script does not account for the extra column.
td_Train_ADS = td_Train_ADS.drop(columns = "sampleid")
td_Train_ADS.to_pandas().head(5)

,cust_id,income,age,tot_cust_years,tot_children,female_ind,single_ind,married_ind,separated_ind,state_code,...,ck_avg_bal,sv_avg_bal,cc_avg_bal,ck_avg_tran_amt,sv_avg_tran_amt,cc_avg_tran_amt,q1_trans_cnt,q2_trans_cnt,q3_trans_cnt,q4_trans_cnt
0,28617939,5724.2,70,10,1,1,0,1,0,CA,...,291.728969,291.728969,291.728969,2.879832,2.879832,2.879832,384,57,69,72
1,27252140,48900.0,44,6,1,0,0,1,0,OTHER,...,4875.089945,0.000000,4875.089945,14.125061,14.125061,14.125061,220,28,30,48
2,25889191,5828.1,69,6,0,1,0,1,0,OTHER,...,0.000000,3568.918000,0.000000,0.000000,0.000000,0.000000,7,9,7,9
3,21806880,0.0,17,5,1,0,1,0,0,TX,...,0.000000,829.489960,829.489960,0.000000,0.000000,37.680260,24,10,6,10
4,23168535,1625.2,39,5,0,1,0,1,0,TX,...,1430.087142,1430.087142,0.000000,41.337208,41.337208,0.000000,80,100,58,22


In [20]:
# Determine the teradataml Script object for the training segment. This object
# will be used to call the SCRIPT Table Operator in the Database. Specify that
# we want to run the training script on data that should be partitioned by the
# state code variable.
# Note: In the present implementation, the output column names are used by the
#       scoring script; handle naming carefully to maintain consistency with
#       code and avoid errors during execution.
#
stoTr = Script(data = td_Train_ADS,
               script_name = "stoRFFitMM.py",
               files_local_path = path_to_files, 
               script_command = "python3 ./TRNG_TECHBYTES/stoRFFitMM.py",
               data_partition_column = "state_code",
               delimiter = ',',
               returns = { "State_Code": VARCHAR(10), "Model": CLOB() }
              )

In [21]:
# If previous versions of files exist in the Database, remove them prior to
# installing the current versions you wish to use.
#
stoTr.remove_file(file_identifier='stoRFFitMM', force_remove=True)

# Install the script and the accompanying model file in the target Advanced SQL
# Engine Database.
#
stoTr.install_file(file_identifier='stoRFFitMM', file_name='stoRFFitMM.py', is_binary=False)

File stoRFFitMM removed from Vantage
File stoRFFitMM.py installed in Vantage


In [22]:
# Execute the script in-Database with the SCRIPT Table Operator.
#
trainOutObj = stoTr.execute_script()
print("STO call complete.")

SELECT * FROM Script(
	ON "ak186064"."ml__select__1620181179851205" AS "input"
	PARTITION BY "state_code"
	
	SCRIPT_COMMAND('python3 ./TRNG_TECHBYTES/stoRFFitMM.py')
	delimiter(',')
	returns('State_Code VARCHAR(10), Model CLOB')
) as sqlmr
STO call complete.


In [23]:
# The trained models are now pointed to by the trainOutObj object.
#
# In the present illustration, we show how to use multipe trained models by
# bringing them locally to the client and then uploading them as a file to
# the Database. A different approach is to store the trained models directly
# into a table in the Database. The latter approach is exhibited in the
# section "Using DataFrame.map_partition() Function for GLM Model Fitting and
# Scoring Functions" of the teradataml User Guide at docs.teradata.com.
#
# Save the trained models in a local csv file. The scoring script will need
# this file to select and use the appropriate model with the corresponding 
# state code partition of test data on each Database AMP.
#
# To save the models into a file on the client, we first bring them locally
# by converting the teradataml DataFrame to a pandas DataFrame.
#
multipleModels = trainOutObj.to_pandas()
multipleModels.head()

,State_Code,Model
0,CA,b'gANjc2tsZWFybi5lbnNlbWJsZS5fZm9yZXN0ClJhbmR...
1,OH,b'gANjc2tsZWFybi5lbnNlbWJsZS5fZm9yZXN0ClJhbmR...
2,OTHER,b'gANjc2tsZWFybi5lbnNlbWJsZS5fZm9yZXN0ClJhbmR...
3,TX,b'gANjc2tsZWFybi5lbnNlbWJsZS5fZm9yZXN0ClJhbmR...
4,AZ,b'gANjc2tsZWFybi5lbnNlbWJsZS5fZm9yZXN0ClJhbmR...


In [24]:
# Then we simply export the pandas DataFrame to a csv file.
# Note: Exclude the DataFrame index from the csv file; this prevents the index
#       from being assumed by the Database to be an additional data column in
#       the csv file. 
#
multipleModels.to_csv(path_to_files + 'multipleModels_py.csv', index = False)

#### 2.2. Models scoring with the STO

In [25]:
# Step 2: Scoring with Models
# 
# Specify the teradataml DataFrame to use with the teradataml Script object.
#
td_Test_ADS = DataFrame("ak_TBv2_Test_ADS_Py")

# Recall that the testing subset table has an additional column with the sample
# ID. We drop it because our script does not account for the extra column.
#
td_Test_ADS = td_Test_ADS.drop(columns = "sampleid")
td_Test_ADS.to_pandas().head(5)

,cust_id,income,age,tot_cust_years,tot_children,female_ind,single_ind,married_ind,separated_ind,state_code,...,ck_avg_bal,sv_avg_bal,cc_avg_bal,ck_avg_tran_amt,sv_avg_tran_amt,cc_avg_tran_amt,q1_trans_cnt,q2_trans_cnt,q3_trans_cnt,q4_trans_cnt
0,20437290,16051.5,81,8,2,1,0,0,1,TX,...,7433.737500,7433.737500,0.000000,-55.206000,-55.206000,0.000000,24,20,26,30
1,21803664,0.0,13,8,1,0,1,0,0,NY,...,0.000000,515.481000,0.000000,0.000000,0.000000,0.000000,3,3,3,3
2,25889191,5828.1,69,6,0,1,0,1,0,OTHER,...,0.000000,3568.918000,0.000000,0.000000,0.000000,0.000000,7,9,7,9
3,21806880,0.0,17,5,1,0,1,0,0,TX,...,0.000000,829.489960,829.489960,0.000000,0.000000,37.680260,24,10,6,10
4,23171187,64076.1,60,6,0,0,0,0,0,NY,...,3751.930591,3751.930591,3751.930591,13.142299,13.142299,13.142299,213,213,96,60


In [26]:
# Determine the teradataml Script object. This object will be used with calls
# to the SCRIPT Table Operator in the Database, as well as with STO script 
# testing in the STO Sandbox.
# Note: In present use case, the Python script will be importing a model file.
#       Remember to adjust the relative model file location in the Python code.
#       In the Database, the model file is expected to be found inside the
#       node directory that is named after the SEARCHUIFDBPATH.
#
stoSc = Script(data = td_Test_ADS,
               script_name = "stoRFScoreMM.py",
               files_local_path = path_to_files, 
               script_command = "python3 ./TRNG_TECHBYTES/stoRFScoreMM.py",
               data_partition_column = "state_code",
               delimiter = ',',
               returns = { "State_Code": VARCHAR(10), "Cust_ID": INTEGER(), 
                           "Prob_0": FLOAT(), "Prob_1": FLOAT(), "Actual": INTEGER() }
              )

In [27]:
# If previous versions of files exist in the Database, remove them prior to
# installing the current versions you wish to use.
#
#stoSc.remove_file(file_identifier='multipleModels_py', force_remove=True)
stoSc.remove_file(file_identifier='stoRFScoreMM', force_remove=True)

# Install the script and the accompanying model file in the target Advanced SQL
# Engine Database.Remember to specify in your script the correct path to the
# model file in the Database: Your script needs to look for the script file in
# a directory named after the SEARCHUIFDBPATH.
#
#stoSc.install_file(file_identifier='multipleModels_py', file_name='multipleModels_py.csv', is_binary=False)
stoSc.install_file(file_identifier='stoRFScoreMM', file_name='stoRFScoreMM.py', is_binary=False)

File stoRFScoreMM removed from Vantage
File stoRFScoreMM.py installed in Vantage


In [28]:
# Execute the script in-Database with the SCRIPT Table Operator.
#
stoSc.execute_script()

SELECT * FROM Script(
	ON "ak186064"."ml__select__1620189650132464" AS "input"
	PARTITION BY "state_code"
	
	SCRIPT_COMMAND('python3 ./TRNG_TECHBYTES/stoRFScoreMM.py')
	delimiter(',')
	returns('State_Code VARCHAR(10), Cust_ID INTEGER, Prob_0 FLOAT, Prob_1 FLOAT, Actual INTEGER')
) as sqlmr


  State_Code   Cust_ID  Prob_0  Prob_1  Actual
0        AZ   27256760   1.000   0.000       0
1        AZ   23164302   0.024   0.976       1
2        AZ   19076484   0.010   0.990       1
3        AZ   19086130   0.000   1.000       1
4        AZ   14992912   0.996   0.004       0
5        AZ   19086816   0.000   1.000       1
6        CA   31358683   0.000   1.000       1
7        CA   24532398   0.000   1.000       1
8        CA   27253380   1.000   0.000       0
9        CA   19075014   0.010   0.990       1

### 3. Map functions
A different way to execute row-based or partition-based operation in the Advanced SQL Engine is via the teradataml DataFrame map_row() and map_partition() methods, respectively. In essence, these methods streamline the set-up and calls to the STO in the background, thus making interaction with the STO transparent to the Python user.

**Caution:** For map_row() and map_partition() to work, teradataml requires the Python _dill_ add-on library version to be same on both the client and the target Advanced SQL Engine.

In the following segment, we present brief examples of the map_row() and map_partition() methods. An additional micromodeling use case example with map_partition() can be found in the section "Using DataFrame.map_partition() Function for GLM Model Fitting and Scoring Functions" of the teradataml User Guide at docs.teradata.com. 

In [29]:
from teradataml import load_example_data
from collections import OrderedDict

# This example uses the 'admissions_train' dataset, and calculates the average
# 'gpa' per partition based on the value in 'admitted' column.
#
# Load the example data. Observe that the load_example_data() function creates
# internal/temp tables, and for this reason it places them in the teradataml 
# context "temp_database_name" database, which presently is "ak186064".
# However, the teradataml DataFrame() function looks by default in the
# teradataml context "database". Therefore, to create a teradataml DataFrame
# in this case, the DataFrame() function must be pointed explicitly to look
# into the "ak186064" database, as shown in the following.
#  
load_example_data("dataframe", "admissions_train")
dfMap = DataFrame(in_schema('ak186064', 'admissions_train'))
dfMap.to_pandas().head(5)

,masters,gpa,stats,programming,admitted
id,,,,,
24,no,1.87,Advanced,Novice,1
27,yes,3.96,Advanced,Advanced,0
29,yes,4.00,Novice,Beginner,0
30,yes,3.79,Advanced,Novice,0
38,yes,2.65,Advanced,Beginner,1


In [30]:
# A user defined function to increase the 'gpa' by a specified percentage.
# Both the function input and output are pandas Series objects.
#
def increase_gpa(row, p = 20):
    row['gpa'] = row['gpa'] + row['gpa'] * p/100
    return row

# Apply the user defined function to the teradataml DataFrame. The output of 
# the user defined function expects the same columns with the same types as
# the input, hence the 'returns' argument of map_row() can be skipped.
#
increase_gpa_10 = dfMap.map_row(lambda row: increase_gpa(row, p = 10))
#
# Note: map_row() can ve also called with only the user-defined function as an
#       argment; alternatively, it can be invoked with partial notation, too:
#       increase_gpa_40 = df.map_row(lambda row: increase_gpa(row, p = 40))
#       from functools import partial
#       increase_gpa_50 = df.map_row(partial(increase_gpa, p = 50))
#
increase_gpa_10.to_pandas().head(5)

SELECT * FROM Script(
	ON "ak186064"."admissions_train" AS "input"
	PARTITION BY ANY
	
	SCRIPT_COMMAND('python3 ./TRNG_TECHBYTES/ml__map_row_1620181259713804.py')
	delimiter('	')
	returns('id INTEGER, masters VARCHAR(5) CHAR SET LATIN, gpa FLOAT, stats VARCHAR(30) CHAR SET LATIN, programming VARCHAR(30) CHAR SET LATIN, admitted INTEGER')
) as sqlmr


,masters,gpa,stats,programming,admitted
id,,,,,
24,no,2.057,Advanced,Novice,1
27,yes,4.356,Advanced,Advanced,0
29,yes,4.400,Novice,Beginner,0
30,yes,4.169,Advanced,Novice,0
38,yes,2.915,Advanced,Beginner,1


In [31]:
# A user defined function to calculate the average 'gpa', by reading data at
# once into a pandas DataFrame. The function accepts a TextFileReader object
# for data iteration in chunks. The function returns a pandas Series.
#
def grouped_gpa_avg(rows):
    pdf = rows.read()
    if pdf.shape[0] > 0:
        return pdf[['admitted', 'gpa']].mean()

# Apply the user defined function to the DataFrame.
#
avg_gpa_pdf = dfMap.map_partition(
                        grouped_gpa_avg,
                        returns = OrderedDict([('admitted', INTEGER()), ('avg_gpa', FLOAT())]),
                        data_partition_column = 'admitted'
                                 )
avg_gpa_pdf.to_pandas()

SELECT * FROM Script(
	ON "ak186064"."admissions_train" AS "input"
	PARTITION BY "admitted"
	
	SCRIPT_COMMAND('python3 ./TRNG_TECHBYTES/ml__map_partition_1620189359538926.py')
	delimiter('	')
	returns('admitted INTEGER, avg_gpa FLOAT')
) as sqlmr


,avg_gpa
admitted,
1,3.533462
0,3.557143


### End of session

In [32]:
# Remove the context of present teradataml session and terminate the Python
# session. It is recommended to call the remove_context() function for session
# cleanup. Temporary objects are removed at the end of the session.
#
remove_context()

True